# Code Github

## Training

In [1]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_from_disk, Dataset
from peft import LoraConfig, get_peft_model

def explode_dataset(dataset):
    """
    Explodes the "text" list in each example so that each element becomes its own example.
    """
    new_examples = []
    for example in dataset:
        # example["text"] is expected to be a list
        for t in example["text"]:
            new_examples.append({"text": t})
    return Dataset.from_list(new_examples)

def main():
    model_name = "mistralai/Mistral-7B-Instruct-v0.3"

    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_4bit=True,
        device_map="auto",
        torch_dtype=torch.float16
    )
    model.config.pretraining_tp = 1
    model.config.window = 256

    lora_config = LoraConfig(
        r=16,
        lora_alpha=32,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=0.1,
        task_type="CAUSAL_LM", 
        bias="none"
    )

    model = get_peft_model(model, lora_config)

    raw_datasets = load_from_disk('E:\\fine_tuning\\dataset_code')

    system_prompt = (
        "\n**Code Generation Request**\n\n"
        " * Read the provided **Method Descriptions & Summaries**\n"
        " * **Complete the Body of Each Block code** based on the corresponding summaries\n"
        " * **Format Requirement:** Wrap all generated **Code Blocks** in triple backticks (```) for enhanced readability\n"
        " * **Delivery Note:** Provide only the completed **Code Blocks** without explanatory comments or text\n"
    )

    def chunk_text(text, tokenizer, max_length=1024, stride=512):
        """
        Tokenize the full text and then split it into overlapping chunks.
        Each chunk is decoded back to a string.
        """
        tokens = tokenizer.encode(text, add_special_tokens=False)
        chunks = []
        for i in range(0, len(tokens), stride):
            chunk_tokens = tokens[i: i + max_length]
            chunk_str = tokenizer.decode(chunk_tokens, skip_special_tokens=True)
            chunks.append(chunk_str)
        return chunks

    def preprocess_example(batch):
        """
        Process a batch of examples by creating sliding window chunks.
        Each example's text is transformed into a list of chunk strings.
        """
        all_chunks = []
        for prompt, response in zip(batch["prompt"], batch["response"]):
            if isinstance(prompt, list):
                prompt = " ".join(prompt)
            if isinstance(response, list):
                response = " ".join(response)
            
            full_text = (
                "<s>[INST] System: " + system_prompt + "\n"
                "User: " + prompt.strip() + " [/INST] \n"
                "Assistant: " + response.strip() + " </s>"
            )
            
            chunks = chunk_text(full_text, tokenizer, max_length=1024, stride=512)
            all_chunks.append(chunks)
        
        return {"text": all_chunks}

    # Process examples in batches.
    processed_datasets = raw_datasets.map(preprocess_example, batched=True, num_proc=4)

    # Explode the "text" list into individual examples for each split.
    for split in processed_datasets.keys():
        processed_datasets[split] = explode_dataset(processed_datasets[split])

    def tokenize_function(example):
        return tokenizer(example["text"], truncation=True, max_length=1024, padding="max_length")

    # Remove unnecessary columns.
    remove_columns = processed_datasets["train"].column_names
    tokenized_datasets = processed_datasets.map(
        tokenize_function,
        batched=True,
        remove_columns=remove_columns,
        num_proc=4
    )

    print("Train dataset size:", len(tokenized_datasets["train"]))
    print("Validation dataset size:", len(tokenized_datasets["validation"]))
    if len(tokenized_datasets["validation"]) > 0:
        print("A sample validation example (tokenized):")
        print(tokenized_datasets["validation"][0])
    else:
        print("Validation dataset is empty!")

    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    save_dir = "./mistral_finetuned_v4"
    print("Saving output to:", save_dir)
    
    training_args = TrainingArguments(
        output_dir=save_dir,
        evaluation_strategy="steps",
        logging_first_step=True,
        eval_steps=1000,
        logging_steps=100,
        num_train_epochs=3,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        fp16=True,
        save_steps=100,
        save_total_limit=2,
        push_to_hub=False,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        data_collator=data_collator,
    )

    trainer.train()

    eval_metrics = trainer.evaluate()
    print("Evaluation metrics:", eval_metrics)

    model.save_pretrained(save_dir + "_lora")

if __name__ == "__main__":
    main()


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Map (num_proc=4):   0%|          | 0/9641 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2488 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2904 [00:00<?, ? examples/s]

Train dataset size: 9641
Validation dataset size: 2488
A sample validation example (tokenized):
{'input_ids': [1, 2903, 29515, 29473, 781, 1116, 3308, 27570, 12284, 1116, 781, 781, 1166, 5707, 1040, 4625, 1387, 4710, 3303, 17903, 1335, 7695, 4247, 1265, 1116, 781, 1166, 1387, 15696, 1040, 17018, 1070, 7834, 9831, 3464, 1116, 3586, 1124, 1040, 8374, 18830, 1265, 781, 1166, 1387, 5824, 8019, 11779, 4817, 1162, 2010, 1312, 7906, 1387, 3308, 9831, 29481, 1116, 1065, 22980, 1620, 29475, 6214, 1093, 14708, 26688, 1122, 22501, 1988, 3205, 781, 1166, 1387, 10257, 7966, 6864, 4817, 7901, 1315, 1633, 1040, 8136, 1387, 3308, 9831, 29481, 1116, 2439, 11696, 6145, 8384, 1210, 3013, 781, 781, 1498, 29515, 2722, 1291, 14915, 29535, 781, 4080, 8670, 29491, 29523, 1349, 29478, 29491, 8055, 1056, 1155, 1619, 5540, 1117, 8100, 1122, 8269, 19507, 1066, 9362, 1789, 9299, 1065, 1040, 28572, 29478, 5761, 11550, 29491, 781, 4080, 2635, 14915, 29535, 781, 4080, 1291, 7286, 29535, 781, 4080, 28572, 29478, 3409,

e:\fine_tuning\env\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
e:\fine_tuning\env\lib\site-packages\bitsandbytes\nn\modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss,Validation Loss
1000,0.550000,0.558384
2000,0.532100,0.544873
3000,0.472300,0.549312
4000,0.457500,0.543884
5000,0.405200,0.556317
6000,0.389100,0.557123
7000,0.386000,0.558952


e:\fine_tuning\env\lib\site-packages\peft\utils\other.py:716: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 3f1889a7-c2a6-4cb2-b6c9-94ec3f5d702e)') - silently ignoring the lookup for the file config.json in mistralai/Mistral-7B-Instruct-v0.3.
  warnings.warn(
e:\fine_tuning\env\lib\site-packages\peft\utils\save_and_load.py:246: UserWarning: Could not find a config file in mistralai/Mistral-7B-Instruct-v0.3 - will assume that the vocabulary was not modified.
  warnings.warn(
e:\fine_tuning\env\lib\site-packages\peft\utils\other.py:716: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 0d2ebc84-e49c-47c9-a01a-d75fdaf3b605)') - silently ignoring the lookup for the file config.json in mistralai/Mistral-7B-I

Evaluation metrics: {'eval_loss': 0.5587656497955322, 'eval_runtime': 5821.9496, 'eval_samples_per_second': 0.427, 'eval_steps_per_second': 0.053, 'epoch': 2.9990664868789545}


In [2]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_from_disk
from peft import LoraConfig, get_peft_model, PeftModel  # Added PeftModel import

def main():
    # [Previous code remains identical until after model.save_pretrained()...]

    save_dir = "./mistral_finetuned_v4"

    # New section: Merge LoRA with base model
    # --------------------------------------------------
    print("\nMerging LoRA adapters with base model...")
    model_name = "mistralai/Mistral-7B-Instruct-v0.3"
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    # Reload base model in FP16 (without 4-bit quantization)
    base_model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype=torch.float16
    )
    
    # Load LoRA adapter weights
    lora_model = PeftModel.from_pretrained(
        base_model,
        save_dir+"_lora",
        torch_dtype=torch.float16
    )
    
    # Merge and save
    merged_model = lora_model.merge_and_unload()
    merged_model.save_pretrained(save_dir+"-merged")
    tokenizer.save_pretrained(save_dir+"-merged")
    
    print("Merged model saved to"+ save_dir +"-merged")
    # --------------------------------------------------

if __name__ == "__main__":
    main()


Merging LoRA adapters with base model...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Some parameters are on the meta device because they were offloaded to the cpu.
e:\fine_tuning\env\lib\site-packages\transformers\modeling_utils.py:2810: UserWarning: Attempting to save a model with offloaded modules. Ensure that unallocated cpu memory exceeds the `shard_size` (5GB default)
  warnings.warn(


Saving checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Merged model saved to./mistral_finetuned_v4-merged


## Test

### Code Bleu

In [1]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_from_disk
from codebleu import calc_codebleu

def main():
    # Specify the model path from Hugging Face
    model_name = "E:\\fine_tuning\\Mistral_v3_instruct\\mistral_finetuned_v4-merged"

    # Load the tokenizer and set the padding token (using eos_token if not already set)
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    # Adjust the tokenizer padding side to "right"
    tokenizer.padding_side = "right"

    # Load the base model in 4-bit mode using bitsandbytes for memory efficiency
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_4bit=True,      # requires bitsandbytes; lowers memory usage
        device_map="auto",
        torch_dtype=torch.float16
    )
    # Additional model configurations
    model.config.pretraining_tp = 1
    model.config.window = 256

    # Load test dataset and select only 10 examples
    full_test_dataset = load_from_disk("E:\\fine_tuning\\dataset_code")["test"]
    # test_dataset = full_test_dataset.select(range(10))
    test_dataset = full_test_dataset
    # Define system prompt
    system_prompt = '''
        **Code Generation Request** 

        * Read the provided **Method Descriptions & Summaries**
        * **Complete the Body of Each Block code** based on the corresponding summaries
        * **Format Requirement:** Wrap all generated **Code Blocks** in triple backticks (```) for enhanced readability
        * **Delivery Note:** Provide only the completed **Code Blocks** without explanatory comments or text
    '''

    # Preprocess function to combine the system prompt and user prompt into a single input string
    def preprocess_example(example):
        example["text"] = (
            "<s>[INST] "
            "System: " + system_prompt + "\n"
            "User: " + example["prompt"].strip() + " [/INST] \n"
            "Assistant: "
        )
        return example

    # Preprocess test dataset
    processed_test_dataset = test_dataset.map(preprocess_example)

    # Tokenization function (using truncation to 2048 tokens for compatibility with fine-tuning)
    def tokenize_function(example):
        return tokenizer(example["text"], truncation=True, max_length=2048, padding="max_length")

    # Tokenize dataset
    tokenized_test_dataset = processed_test_dataset.map(
        tokenize_function, 
        batched=True, 
        remove_columns=test_dataset.column_names
    )

    # Generate predictions using max_new_tokens instead of max_length
    def generate_code(example):
        input_ids = tokenizer(example["text"], return_tensors="pt").input_ids.to("cuda")
        output = model.generate(
            input_ids,
            max_new_tokens=1024,  # Generate 512 new tokens after the prompt
            pad_token_id=tokenizer.pad_token_id
        )
        example["generated_code"] = tokenizer.decode(output[0], skip_special_tokens=True)
        return example

    generated_results = tokenized_test_dataset.map(generate_code)

    # Evaluate with CodeBLEU on these 10 examples
    references = [ex["response"] for ex in test_dataset]
    hypotheses = [ex["generated_code"] for ex in generated_results]
    # Compute CodeBLEU score
    codebleu_score = calc_codebleu(
        references,        # list of reference code (or list of lists if there are multiple references)
        hypotheses,        # list of candidate code
        lang="c_sharp",           # specify the programming language
        weights=(0.25, 0.25, 0.25, 0.25),  # weights for n-gram, weighted n-gram, syntax, and data-flow matches
        tokenizer=None           # if None, the default string split is used
    )

    print("CodeBLEU Score for all examples:", codebleu_score)

if __name__ == "__main__":
    main()


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/632 [00:00<?, ? examples/s]

Map:   0%|          | 0/632 [00:00<?, ? examples/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
e:\fine_tuning\env\lib\site-packages\bitsandbytes\nn\modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


CodeBLEU Score for all examples: {'codebleu': 0.2742492493504123, 'ngram_match_score': 0.10951106428853616, 'weighted_ngram_match_score': 0.3116537412315968, 'syntax_match_score': 0.2641481075916413, 'dataflow_match_score': 0.41168408428987496}


### perplaxity

# Code Bank

## Training

In [1]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_from_disk, Dataset
from peft import LoraConfig, get_peft_model

def explode_dataset(dataset):
    """
    Explodes the "text" list in each example so that each element becomes its own example.
    """
    new_examples = []
    for example in dataset:
        # example["text"] is expected to be a list
        for t in example["text"]:
            new_examples.append({"text": t})
    return Dataset.from_list(new_examples)
def main():
    model_name = "E:\\fine_tuning\\Mistral_v3_instruct\\mistral_finetuned_v4-merged"

    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_4bit=True,
        device_map="auto",
        torch_dtype=torch.float16
    )
    model.config.pretraining_tp = 1
    model.config.window = 256

    lora_config = LoraConfig(
        r=16,
        lora_alpha=32,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=0.1,
        task_type="CAUSAL_LM", 
        bias="none"
    )

    model = get_peft_model(model, lora_config)

    raw_datasets = load_from_disk('E:\\fine_tuning\\dataset_code_bank')

    system_prompt = (
        "\n**Code Generation Request**\n\n"
        " * Read the provided **Method Descriptions & Summaries**\n"
        " * **Complete the Body of Each Block code** based on the corresponding summaries\n"
        " * **Format Requirement:** Wrap all generated **Code Blocks** in triple backticks (```) for enhanced readability\n"
        " * **Delivery Note:** Provide only the completed **Code Blocks** without explanatory comments or text\n"
    )

    def chunk_text(text, tokenizer, max_length=1024, stride=512):
        """
        Tokenize the full text and then split it into overlapping chunks.
        Each chunk is decoded back to a string.
        """
        tokens = tokenizer.encode(text, add_special_tokens=False)
        chunks = []
        for i in range(0, len(tokens), stride):
            chunk_tokens = tokens[i: i + max_length]
            chunk_str = tokenizer.decode(chunk_tokens, skip_special_tokens=True)
            chunks.append(chunk_str)
        return chunks

    def preprocess_example(batch):
        """
        Process a batch of examples by creating sliding window chunks.
        Each example's text is transformed into a list of chunk strings.
        """
        all_chunks = []
        for prompt, response in zip(batch["prompt"], batch["response"]):
            if isinstance(prompt, list):
                prompt = " ".join(prompt)
            if isinstance(response, list):
                response = " ".join(response)
            
            full_text = (
                "<s>[INST] System: " + system_prompt + "\n"
                "User: " + prompt.strip() + " [/INST] \n"
                "Assistant: " + response.strip() + " </s>"
            )
            
            chunks = chunk_text(full_text, tokenizer, max_length=1024, stride=512)
            all_chunks.append(chunks)
        
        return {"text": all_chunks}

    # Process examples in batches.
    processed_datasets = raw_datasets.map(preprocess_example, batched=True, num_proc=4)

    # Explode the "text" list into individual examples for each split.
    for split in processed_datasets.keys():
        processed_datasets[split] = explode_dataset(processed_datasets[split])

    def tokenize_function(example):
        return tokenizer(example["text"], truncation=True, max_length=1024, padding="max_length")

    # Remove unnecessary columns.
    remove_columns = processed_datasets["train"].column_names
    tokenized_datasets = processed_datasets.map(
        tokenize_function,
        batched=True,
        remove_columns=remove_columns,
        num_proc=4
    )

    print("Train dataset size:", len(tokenized_datasets["train"]))
    print("Validation dataset size:", len(tokenized_datasets["validation"]))
    if len(tokenized_datasets["validation"]) > 0:
        print("A sample validation example (tokenized):")
        print(tokenized_datasets["validation"][0])
    else:
        print("Validation dataset is empty!")

    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


    save_dir = "./mistral_finetuned_v5"
    
    print(save_dir)
    training_args = TrainingArguments(
        output_dir=save_dir,
        evaluation_strategy="steps",
        logging_first_step=True,
        eval_steps=1000,
        logging_steps=100,
        num_train_epochs=2,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        fp16=True,
        save_steps=100,
        save_total_limit=2,
        push_to_hub=False,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        data_collator=data_collator,
    )

    trainer.train()

    eval_metrics = trainer.evaluate()
    print("Evaluation metrics:", eval_metrics)

    model.save_pretrained(save_dir+"_lora")

if __name__ == "__main__":
    main()


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Map (num_proc=4):   0%|          | 0/10552 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/873 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/996 [00:00<?, ? examples/s]

Train dataset size: 10552
Validation dataset size: 873
A sample validation example (tokenized):
{'input_ids': [1, 2903, 29515, 29473, 781, 1116, 3308, 27570, 12284, 1116, 781, 781, 1166, 5707, 1040, 4625, 1387, 4710, 3303, 17903, 1335, 7695, 4247, 1265, 1116, 781, 1166, 1387, 15696, 1040, 17018, 1070, 7834, 9831, 3464, 1116, 3586, 1124, 1040, 8374, 18830, 1265, 781, 1166, 1387, 5824, 8019, 11779, 4817, 1162, 2010, 1312, 7906, 1387, 3308, 9831, 29481, 1116, 1065, 22980, 1620, 29475, 6214, 1093, 14708, 26688, 1122, 22501, 1988, 3205, 781, 1166, 1387, 10257, 7966, 6864, 4817, 7901, 1315, 1633, 1040, 8136, 1387, 3308, 9831, 29481, 1116, 2439, 11696, 6145, 8384, 1210, 3013, 781, 781, 1498, 29515, 2722, 1291, 14915, 29535, 781, 4080, 7121, 7887, 29491, 27053, 1155, 1619, 5540, 1117, 8100, 1122, 13620, 5754, 2474, 8844, 1065, 1040, 5454, 7854, 29493, 10875, 1122, 2956, 21426, 1072, 3995, 2605, 29491, 781, 4080, 2635, 14915, 29535, 781, 4080, 1291, 7286, 29535, 781, 4080, 1268, 4595, 27053, 11

e:\fine_tuning\env\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
e:\fine_tuning\env\lib\site-packages\bitsandbytes\nn\modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss,Validation Loss
1000,0.153100,0.216416
2000,0.122700,0.190983
3000,0.100600,0.179494
4000,0.100200,0.167556
5000,0.086100,0.162889


Evaluation metrics: {'eval_loss': 0.16202066838741302, 'eval_runtime': 2035.1329, 'eval_samples_per_second': 0.429, 'eval_steps_per_second': 0.054, 'epoch': 2.0}


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import  PeftModel  # Added PeftModel import

def main():
    # [Previous code remains identical until after model.save_pretrained()...]

    save_dir =  "./mistral_finetuned_v5"

    # New section: Merge LoRA with base model
    # --------------------------------------------------
    print("\nMerging LoRA adapters with base model...")
    model_name = "E:\\fine_tuning\\Mistral_v3_instruct\\mistral_finetuned_v4-merged"
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    # Reload base model in FP16 (without 4-bit quantization)
    base_model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype=torch.float16
    )
    
    # Load LoRA adapter weights
    lora_model = PeftModel.from_pretrained(
        base_model,
        save_dir+"_lora",
        torch_dtype=torch.float16
    )
    
    # Merge and save
    merged_model = lora_model.merge_and_unload()
    merged_model.save_pretrained(save_dir+"-merged")
    tokenizer.save_pretrained(save_dir+"-merged")
    
    print("Merged model saved to"+ save_dir +"-merged")
    # --------------------------------------------------

if __name__ == "__main__":
    main()


Merging LoRA adapters with base model...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Some parameters are on the meta device because they were offloaded to the cpu.
e:\fine_tuning\env\lib\site-packages\transformers\modeling_utils.py:2810: UserWarning: Attempting to save a model with offloaded modules. Ensure that unallocated cpu memory exceeds the `shard_size` (5GB default)
  warnings.warn(


Saving checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Merged model saved to./mistral_finetuned_v5-merged


In [4]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_from_disk
from codebleu import calc_codebleu

def main():
    # Specify the model path from Hugging Face
    model_name = "E:\\fine_tuning\\Mistral_v3_instruct\\mistral_finetuned_v5-merged"

    # Load the tokenizer and set the padding token (using eos_token if not already set)
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    # Adjust the tokenizer padding side to "right"
    tokenizer.padding_side = "right"

    # Load the base model in 4-bit mode using bitsandbytes for memory efficiency
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_4bit=True,      # requires bitsandbytes; lowers memory usage
        device_map="auto",
        torch_dtype=torch.float16
    )
    # Additional model configurations
    model.config.pretraining_tp = 1
    model.config.window = 256

    # Load test dataset and select only 10 examples
    full_test_dataset = load_from_disk("E:\\fine_tuning\\dataset_code_bank")["test"]
    # test_dataset = full_test_dataset.select(range(10))
    test_dataset = full_test_dataset
    # Define system prompt
    system_prompt = '''
        **Code Generation Request** 

        * Read the provided **Method Descriptions & Summaries**
        * **Complete the Body of Each Block code** based on the corresponding summaries
        * **Format Requirement:** Wrap all generated **Code Blocks** in triple backticks (```) for enhanced readability
        * **Delivery Note:** Provide only the completed **Code Blocks** without explanatory comments or text
    '''

    # Preprocess function to combine the system prompt and user prompt into a single input string
    def preprocess_example(example):
        example["text"] = (
            "<s>[INST] "
            "System: " + system_prompt + "\n"
            "User: " + example["prompt"].strip() + " [/INST] \n"
            "Assistant: "
        )
        return example

    # Preprocess test dataset
    processed_test_dataset = test_dataset.map(preprocess_example)

    # Tokenization function (using truncation to 2048 tokens for compatibility with fine-tuning)
    def tokenize_function(example):
        return tokenizer(example["text"], truncation=True, max_length=2048, padding="max_length")

    # Tokenize dataset
    tokenized_test_dataset = processed_test_dataset.map(
        tokenize_function, 
        batched=True, 
        remove_columns=test_dataset.column_names
    )

    # Generate predictions using max_new_tokens instead of max_length
    def generate_code(example):
        input_ids = tokenizer(example["text"], return_tensors="pt").input_ids.to("cuda")
        output = model.generate(
            input_ids,
            max_new_tokens=1024,  # Generate 512 new tokens after the prompt
            pad_token_id=tokenizer.pad_token_id
        )
        example["generated_code"] = tokenizer.decode(output[0], skip_special_tokens=True)
        return example

    generated_results = tokenized_test_dataset.map(generate_code)

    # Evaluate with CodeBLEU on these 10 examples
    references = [ex["response"] for ex in test_dataset]
    hypotheses = [ex["generated_code"] for ex in generated_results]
    # Compute CodeBLEU score
    codebleu_score = calc_codebleu(
        references,        # list of reference code (or list of lists if there are multiple references)
        hypotheses,        # list of candidate code
        lang="c_sharp",           # specify the programming language
        weights=(0.25, 0.25, 0.25, 0.25),  # weights for n-gram, weighted n-gram, syntax, and data-flow matches
        tokenizer=None           # if None, the default string split is used
    )

    print("CodeBLEU Score for all examples:", codebleu_score)

if __name__ == "__main__":
    main()


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/147 [00:00<?, ? examples/s]

CodeBLEU Score for all examples: {'codebleu': 0.2655661612783676, 'ngram_match_score': 0.1040344903212302, 'weighted_ngram_match_score': 0.21692194150853925, 'syntax_match_score': 0.6176635591122844, 'dataflow_match_score': 0.12364465417141646}
